<a href="https://colab.research.google.com/github/efremfilho/python_scripts/blob/master/test_agentKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-agents
!pip install openai
!pip install pydantic

In [7]:
from pydantic import BaseModel
from agents import Agent, ModelSettings, TResponseInputItem, Runner, RunConfig
from openai.types.shared.reasoning import Reasoning

class TriageRequestSchema(BaseModel):
  classification: str


class ApprovalAgentSchema(BaseModel):
  emailFrom: str
  defaultTo: str
  defaultSubject: str
  defaultBody: str


triage_request = Agent(
  name="Triage request",
  instructions="""Classify the user's request based on whether two documents have been provided recently in the conversation, and whether the user is asking a particular question.

If two documents are provided and there's no user question , respond with \"compare\".
If two documents are provided and there is a user question , respond with \"answer_question\".
If only one doc has been provided, or no docs have been provided, respond with \"request_upload\"""",
  model="gpt-4.1",
  output_type=TriageRequestSchema,
  model_settings=ModelSettings(
    temperature=1,
    top_p=1,
    max_tokens=2048,
    store=True
  )
)


propose_reconciliation = Agent(
  name="Propose reconciliation",
  instructions="Given the differences between the two documents, assemble a single option for how to reconcile the difference. If no order has been described, consider the first document the user's version and the second document the potential set of changes returned back to the user. The proposal you create will be sent to the user for approval.",
  model="gpt-5",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="minimal",
      summary="auto"
    )
  )
)


approval_agent = Agent(
  name="Approval agent",
  instructions="""Explain your approval reasoning. Help the user draft a proper response by filling out this data schema:

{
  emailFrom: 'user@test.com',
  defaultTo: 'user@test.com',
  defaultSubject: 'Document comparison proposal',
  defaultBody: \"Hey there, \n\nHope you're doing well! Just wanted to check in and see if there are any updates on the ChatKit roadmap. We're excited to see what's coming next and how we can make the most of the upcoming features.\n\nEspecially curious to see how you support widgets!\n\nBest,\",
}""",
  model="gpt-5-mini",
  output_type=ApprovalAgentSchema,
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="low",
      summary="auto"
    )
  )
)


rejection_agent = Agent(
  name="Rejection agent",
  instructions="Explain your rejection reasoning.",
  model="gpt-5",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="low",
      summary="auto"
    )
  )
)


retry_agent = Agent(
  name="Retry agent",
  instructions="The user has not uploaded the required two documents for comparison. Suggest that they upload a total of two documents, using the paperclip icon.",
  model="gpt-5-nano",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="minimal",
      summary="auto"
    )
  )
)


provide_explanation = Agent(
  name="Provide explanation",
  instructions="Use the information in the uploaded documents to answer the user's question.",
  model="gpt-5-nano",
  model_settings=ModelSettings(
    store=True,
    reasoning=Reasoning(
      effort="minimal",
      summary="auto"
    )
  )
)


def approval_request(message: str):
  # TODO: Implement
  return True

class WorkflowInput(BaseModel):
  input_as_text: str


# Main code entrypoint
async def run_workflow(workflow_input: WorkflowInput):
  state = {

  }
  workflow = workflow_input.model_dump()
  conversation_history: list[TResponseInputItem] = [
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": workflow["input_as_text"]
        }
      ]
    }
  ]
  triage_request_result_temp = await Runner.run(
    triage_request,
    input=[
      *conversation_history
    ],
    run_config=RunConfig(trace_metadata={
      "__trace_source__": "agent-builder",
      "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
    })
  )

  conversation_history.extend([item.to_input_item() for item in triage_request_result_temp.new_items])

  triage_request_result = {
    "output_text": triage_request_result_temp.final_output.json(),
    "output_parsed": triage_request_result_temp.final_output.model_dump()
  }
  if triage_request_result["output_parsed"]["classification"] == "compare":
    propose_reconciliation_result_temp = await Runner.run(
      propose_reconciliation,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
      })
    )

    conversation_history.extend([item.to_input_item() for item in propose_reconciliation_result_temp.new_items])

    propose_reconciliation_result = {
      "output_text": propose_reconciliation_result_temp.final_output_as(str)
    }
    approval_message = f"Please review the proposal {propose_reconciliation_result["output_text"]}"

    if approval_request(approval_message):
        approval_agent_result_temp = await Runner.run(
          approval_agent,
          input=[
            *conversation_history
          ],
          run_config=RunConfig(trace_metadata={
            "__trace_source__": "agent-builder",
            "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
          })
        )

        conversation_history.extend([item.to_input_item() for item in approval_agent_result_temp.new_items])

        approval_agent_result = {
          "output_text": approval_agent_result_temp.final_output.json(),
          "output_parsed": approval_agent_result_temp.final_output.model_dump()
        }
    else:
        rejection_agent_result_temp = await Runner.run(
          rejection_agent,
          input=[
            *conversation_history
          ],
          run_config=RunConfig(trace_metadata={
            "__trace_source__": "agent-builder",
            "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
          })
        )

        conversation_history.extend([item.to_input_item() for item in rejection_agent_result_temp.new_items])

        rejection_agent_result = {
          "output_text": rejection_agent_result_temp.final_output_as(str)
        }
  elif triage_request_result["output_parsed"]["classification"] == "answer_question":
    provide_explanation_result_temp = await Runner.run(
      provide_explanation,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
      })
    )

    conversation_history.extend([item.to_input_item() for item in provide_explanation_result_temp.new_items])

    provide_explanation_result = {
      "output_text": provide_explanation_result_temp.final_output_as(str)
    }
  else:
    retry_agent_result_temp = await Runner.run(
      retry_agent,
      input=[
        *conversation_history
      ],
      run_config=RunConfig(trace_metadata={
        "__trace_source__": "agent-builder",
        "workflow_id": "wf_68e7d3ecffdc81909d6bd4ef54e13f97041f23f4d1d6d373"
      })
    )

    conversation_history.extend([item.to_input_item() for item in retry_agent_result_temp.new_items])

    retry_agent_result = {
      "output_text": retry_agent_result_temp.final_output_as(str)
    }


In [10]:
import asyncio
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time

# --- ETAPA 1: IMPORTAR AS FUNÇÕES E CLASSES ORIGINAIS DO BLOCO ANTERIOR ---
# Esta é a parte chave: trazemos as definições originais para poder substituí-las.
# Usamos __main__ para acessar o escopo global do notebook.
from __main__ import run_workflow as original_run_workflow
from __main__ import approval_request as original_approval_request
from __main__ import WorkflowInput, TResponseInputItem # E as classes que precisamos

# --- ETAPA 2: CONFIGURAÇÃO DA INTERFACE E LÓGICA DE APROVAÇÃO ---

# Variável global para controlar a pausa/continuação
approval_future = None

# Widgets da Interface Principal (sem alterações)
text_input = widgets.Textarea(placeholder='Digite sua requisição aqui...', description='Input:', layout=widgets.Layout(width='auto', height='100px'))
file_upload = widgets.FileUpload(accept='.pdf,.txt,.md', multiple=True, description='Upload Arquivos:')
run_button = widgets.Button(description="Executar Workflow", button_style='success', icon='rocket')
main_output_area = widgets.Output()

# Widgets da Interface de Aprovação (sem alterações)
approval_output_area = widgets.Output()
approval_message_label = widgets.Label()
approve_button = widgets.Button(description="Aprovar", button_style='success', icon='check')
reject_button = widgets.Button(description="Rejeitar", button_style='danger', icon='times')
approval_box = widgets.VBox([widgets.HTML("<b>👇 Ação necessária:</b>"), approval_message_label, widgets.HBox([approve_button, reject_button])])

def on_approve_clicked(b):
    global approval_future
    if approval_future and not approval_future.done():
        approval_future.set_result(True)
    with approval_output_area:
        clear_output()

def on_reject_clicked(b):
    global approval_future
    if approval_future and not approval_future.done():
        approval_future.set_result(False)
    with approval_output_area:
        clear_output()

approve_button.on_click(on_approve_clicked)
reject_button.on_click(on_reject_clicked)


# --- ETAPA 3: "MONKEY PATCHING" - REDEFININDO AS FUNÇÕES ---

def patched_approval_request(message: str) -> bool:
    """
    Esta é a nossa versão síncrona (blocking) da função de aprovação.
    Ela exibe os botões e espera em um loop até que o usuário clique.
    """
    global approval_future
    # Usa o loop de eventos asyncio para criar a Future
    approval_future = asyncio.get_event_loop().create_future()
    approval_message_label.value = message
    with approval_output_area:
        display(approval_box)

    # Loop de espera síncrono. Não é ideal, mas funciona neste ambiente
    # e respeita a chamada síncrona no bloco de código original.
    while not approval_future.done():
        time.sleep(0.1)

    return approval_future.result()


async def patched_run_workflow(workflow_input: WorkflowInput, files: dict = None):
    """
    Esta é a nossa versão do workflow que aceita arquivos e usa a
    lógica de aprovação corrigida.
    """
    if files is None:
        files = {}

    # Recriamos o início da função original, mas agora adicionando os arquivos
    workflow = workflow_input.model_dump()
    conversation_history: list[TResponseInputItem] = [
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": workflow["input_as_text"]},
                *[
                    {"type": "file", "name": name, "content": content}
                    for name, content in files.items()
                ]
            ]
        }
    ]

    # Para o resto da lógica, em vez de copiar/colar todo o workflow,
    # uma abordagem mais limpa seria chamar a função original com um
    # estado pré-configurado. Mas para simplicidade e garantia de funcionamento,
    # vamos apenas simular que o restante do código original é executado.
    # A maneira mais robusta é substituir as chamadas dentro do código original.

    # Substituindo a função de aprovação globalmente antes de chamar o workflow original
    __main__.approval_request = patched_approval_request

    # A lógica para popular 'conversation_history' precisa ser injetada.
    # A forma mais segura de fazer isso sem replicar o código é, infelizmente,
    # não possível sem modificar o bloco original.
    # Portanto, a solução mais pragmática é replicar a lógica do workflow aqui.
    # (Este código é uma réplica do seu bloco 2 com as correções)

    from __main__ import Runner, RunConfig, triage_request, propose_reconciliation, approval_agent, rejection_agent, retry_agent, provide_explanation

    triage_request_result_temp = await Runner.run(triage_request, input=[*conversation_history], run_config=RunConfig(trace_metadata={"__trace_source__": "agent-builder"}))
    conversation_history.extend([item.to_input_item() for item in triage_request_result_temp.new_items])
    triage_request_result = {"output_parsed": triage_request_result_temp.final_output.model_dump()}

    if triage_request_result["output_parsed"]["classification"] == "compare":
        propose_reconciliation_result_temp = await Runner.run(propose_reconciliation, input=[*conversation_history], run_config=RunConfig(trace_metadata={"__trace_source__": "agent-builder"}))
        conversation_history.extend([item.to_input_item() for item in propose_reconciliation_result_temp.new_items])
        propose_reconciliation_result = {"output_text": propose_reconciliation_result_temp.final_output_as(str)}
        approval_message = f"Revisão da proposta: {propose_reconciliation_result['output_text']}"

        if patched_approval_request(approval_message): # Usa nossa função
            approval_agent_result_temp = await Runner.run(approval_agent, input=[*conversation_history], run_config=RunConfig(trace_metadata={"__trace_source__": "agent-builder"}))
            print("Resultado da Aprovação:", approval_agent_result_temp.final_output.model_dump())
        else:
            rejection_agent_result_temp = await Runner.run(rejection_agent, input=[*conversation_history], run_config=RunConfig(trace_metadata={"__trace_source__": "agent-builder"}))
            print("Resultado da Rejeição:", rejection_agent_result_temp.final_output_as(str))

    elif triage_request_result["output_parsed"]["classification"] == "answer_question":
        provide_explanation_result_temp = await Runner.run(provide_explanation, input=[*conversation_history], run_config=RunConfig(trace_metadata={"__trace_source__": "agent-builder"}))
        print("Explicação:", provide_explanation_result_temp.final_output_as(str))
    else: # request_upload
        retry_agent_result_temp = await Runner.run(retry_agent, input=[*conversation_history], run_config=RunConfig(trace_metadata={"__trace_source__": "agent-builder"}))
        print("Tente Novamente:", retry_agent_result_temp.final_output_as(str))


# --- ETAPA 4: LÓGICA DE EXECUÇÃO QUE CHAMA NOSSA VERSÃO CORRIGIDA ---

async def on_run_workflow_click(b):
    with main_output_area:
        clear_output(wait=True)
        print("🚀 Iniciando execução do workflow (versão corrigida)...")
        run_button.disabled = True

        workflow_input_text = WorkflowInput(input_as_text=text_input.value)
        uploaded_files_content = {
            filename: file_data['content']
            for filename, file_data in file_upload.value.items()
        }

        try:
            # Chama a NOSSA função, que sabe lidar com arquivos e interação
            await patched_run_workflow(workflow_input_text, files=uploaded_files_content)
            print("\n✅ Workflow concluído!")
        except Exception as e:
            print(f"\n❌ Ocorreu um erro durante a execução: {e}")
        finally:
            run_button.disabled = False

run_button.on_click(lambda b: asyncio.create_task(on_run_workflow_click(b)))

# --- ETAPA 5: EXIBIÇÃO DA UI ---

display(HTML("<h2>Interface de Teste para Workflows AgentKit</h2>"))
display(text_input, file_upload, run_button)
display(HTML("<hr><h4>Saída do Workflow:</h4>"))
display(main_output_area)
display(HTML("<hr><h4>Painel de Aprovação:</h4>"))
display(approval_output_area)

Textarea(value='', description='Input:', layout=Layout(height='100px', width='auto'), placeholder='Digite sua …

FileUpload(value={}, accept='.pdf,.txt,.md', description='Upload Arquivos:', multiple=True)

Button(button_style='success', description='Executar Workflow', icon='rocket', style=ButtonStyle())

Output()

Output()